# FNN housing example
SGD 기준

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['target'] = boston.target

In [4]:
print(df.shape)
df.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
# Keras에서 불러오는 데이터의 train test index는 다를 수 있음
from sklearn.model_selection import train_test_split
train_data, test_data, train_targets, test_targets = train_test_split(df.values[:,:-1], df.values[:,-1], test_size=0.2, random_state=42)

In [6]:
# z-score normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [81]:
## original tensorflow codes
# import tensorflow as tf
# from tensorflow.keras import layers, models
# model = models.Sequential()
# model.add(layers.Dense(64, activation='relu', kernel_initializer='glorot_uniform',input_shape=(train_data.shape[1],)))
# model.add(layers.Dense(64, activation='relu', kernel_initializer='glorot_uniform'))
# model.add(layers.Dense(1))

In [15]:
# model
class MLP_Net(nn.Module):
    def __init__(self, input_size, hidden_size1=64, hidden_size2=64, output_size=1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.activation = nn.ReLU()
        self.initialize = torch.nn.init.xavier_uniform_

        self.initialize(self.fc1.weight.data)
        self.initialize(self.fc2.weight.data)
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        return x

input_size = train_data.shape[1]
model = MLP_Net(input_size=input_size)

In [16]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        super().__init__()
        self.x = x_tensor
        self.y = y_tensor
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)
        
train_data = torch.Tensor(train_data)
train_targets = torch.Tensor(train_targets)
test_data = torch.Tensor(test_data)
test_targets = torch.Tensor(test_targets)
train_dataset = CustomDataset(train_data, train_targets)
test_dataset = CustomDataset(test_data, test_targets)


In [17]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

epochs = 80
# 다른 optimizer 사용 가능
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)
# optimizer = optim.Adam(model.parameters(),lr=0.001)
criterion = nn.MSELoss().to(device)
# criterion = F.mse_loss().to(device)

In [18]:
model.train()
# losses = []
for epoch in range(epochs):
    train_losses = 0
    data_count = 0
    for i,(inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        optimizer.zero_grad()
        loss = criterion(outputs, targets.view(-1,1))
        loss.backward()
        optimizer.step()
        data_count += len(targets)
        # losses.append(loss.item())
        train_losses += (loss.item() * len(targets))
    train_losses /= data_count
    print(f'Epoch: {epoch}/{epochs}, train_loss = {train_losses:.4f}')
    

Epoch: 0/80, train_loss = 506.4531
Epoch: 1/80, train_loss = 86.5718
Epoch: 2/80, train_loss = 27.6986
Epoch: 3/80, train_loss = 17.9663
Epoch: 4/80, train_loss = 15.8619
Epoch: 5/80, train_loss = 13.8324
Epoch: 6/80, train_loss = 11.9657
Epoch: 7/80, train_loss = 11.7827
Epoch: 8/80, train_loss = 12.1598
Epoch: 9/80, train_loss = 10.1205
Epoch: 10/80, train_loss = 9.5888
Epoch: 11/80, train_loss = 9.2459
Epoch: 12/80, train_loss = 9.5615
Epoch: 13/80, train_loss = 8.4899
Epoch: 14/80, train_loss = 8.4303
Epoch: 15/80, train_loss = 8.3829
Epoch: 16/80, train_loss = 7.9714
Epoch: 17/80, train_loss = 7.7313
Epoch: 18/80, train_loss = 7.5618
Epoch: 19/80, train_loss = 7.6541
Epoch: 20/80, train_loss = 7.2659
Epoch: 21/80, train_loss = 6.8254
Epoch: 22/80, train_loss = 7.2568
Epoch: 23/80, train_loss = 6.8550
Epoch: 24/80, train_loss = 6.5442
Epoch: 25/80, train_loss = 6.6454
Epoch: 26/80, train_loss = 6.7458
Epoch: 27/80, train_loss = 6.2386
Epoch: 28/80, train_loss = 6.2327
Epoch: 29/80,

In [19]:
model.eval()
targets_list = []
outputs_list = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        targets = targets.detach().cpu().numpy()
        outputs = outputs.detach().cpu().numpy()
        targets_list.append(targets)
        outputs_list.append(outputs)
        # sgd.zero_grad()
        # loss.backward()
        # sgd.step()
        # data_count += inputs.shape[0]

In [20]:
from sklearn.metrics import mean_squared_error as mse
mse(np.concatenate(targets_list), np.concatenate(outputs_list))**0.5

3.177858318591625

### 단 skorch를 써서 모델 돌려보면 keras만큼 편리한 API 사용가능

In [111]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

from skorch import NeuralNetRegressor

net = NeuralNetRegressor(
    MLP_Net(input_size=input_size),
    max_epochs=80,
    lr=0.001,
    device=device,
    optimizer=torch.optim.SGD,
    optimizer__momentum=0.9,
    optimizer__nesterov=True,
    batch_size = 64,
    train_split=None  # no validation set split -> fully train on train set
)

In [112]:
# DataSet으로 넣기 가능(y=None)
net.fit(torch.Tensor(train_data), torch.Tensor(train_targets.reshape(-1, 1)))

  epoch    train_loss     dur
-------  ------------  ------
      1      493.7890  0.0162
      2       72.2849  0.0163
      3       24.3399  0.0172
      4       18.3379  0.0174
      5       14.6024  0.0176
      6       12.8560  0.0247
      7       12.1223  0.0209
      8       11.2419  0.0189
      9       10.5558  0.0178
     10       10.0890  0.0176
     11        9.7039  0.0171
     12        9.2995  0.0171
     13        9.0077  0.0163
     14        8.6788  0.0232
     15        8.3675  0.0120
     16        8.0916  0.0133
     17        7.8517  0.0135
     18        7.5930  0.0144
     19        7.3824  0.0149
     20        7.1630  0.0148
     21        6.9772  0.0145
     22        6.7895  0.0144
     23        6.6485  0.0148
     24        6.4882  0.0159
     25        6.3049  0.0163
     26        6.1581  0.0174
     27        6.0036  0.0174
     28        5.8556  0.0177
     29        5.7259  0.0178
     30        5.5922  0.0181
     31        5.4823  0.0177
     32   

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=MLP_Net(
    (fc1): Linear(in_features=13, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=1, bias=True)
    (activation): ReLU()
  ),
)

In [113]:
from sklearn.metrics import mean_squared_error as mse

pred = net.predict(torch.Tensor(test_data))
mse(test_targets, pred)**0.5

3.45181884918875

In [119]:
# 혹은 다음과 같이도 가능
net = NeuralNetRegressor(
    MLP_Net(input_size=input_size),
    max_epochs=80,
    lr=0.001,
    device=device,
    optimizer=torch.optim.SGD,
    optimizer__momentum=0.9,
    optimizer__nesterov=True,
    batch_size = 64,
    train_split=None  # no validation set split -> fully train on train set
)
from torch.utils.data import TensorDataset
temp = TensorDataset(torch.Tensor(train_data), torch.Tensor(train_targets.reshape(-1, 1)))
net.fit(temp, y=None)

  epoch    train_loss     dur
-------  ------------  ------
      1      533.8962  0.0176
      2       90.1277  0.0165
      3       27.6133  0.0166
      4       17.7880  0.0171
      5       14.6904  0.0172
      6       12.8080  0.0284
      7       12.0846  0.0101
      8       11.1892  0.0111
      9       10.5502  0.0126
     10       10.0969  0.0136
     11        9.7175  0.0137
     12        9.3291  0.0140
     13        9.0531  0.0145
     14        8.7873  0.0150
     15        8.5502  0.0147
     16        8.3568  0.0153
     17        8.0991  0.0151
     18        7.9316  0.0154
     19        7.6991  0.0143
     20        7.5003  0.0150
     21        7.3385  0.0150
     22        7.1672  0.0160
     23        7.0034  0.0146
     24        6.8821  0.0149
     25        6.7103  0.0147
     26        6.6248  0.0151
     27        6.4927  0.0154
     28        6.3634  0.0147
     29        6.2499  0.0179
     30        6.1482  0.0244
     31        6.0159  0.0206
     32   

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=MLP_Net(
    (fc1): Linear(in_features=13, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=1, bias=True)
    (activation): ReLU()
  ),
)